# Project Euler problem 178 - Step Numbers

[Link to problem on Project Euler homepage](https://projecteuler.net/problem=178)

## Desciption

Consider the number $45656$.

It can be seen that each pair of consecutive digits of $45656$ has a difference of one. A number for which every pair of consecutive digits has a difference of one is called a step number. A pandigital number contains every decimal digit from $0$ to $9$ at least once. How many pandigital step numbers less than $10^{40}$ are there?

## Solution

It is quite simple to create a recursive algorithm that counts all step numbers up to a given length (expressed as the max number of digits).

In [1]:
def count_step_numbers(start, length=10):
    """`start` is starting digit
    `length` is max number of digits"""
    n_step = 1
    if length == 1: 
        return n_step # exactly one step number with length one
    if start > 0:
        n_step += count_step_numbers(start-1, length=length-1)
    if start < 9:
        n_step += count_step_numbers(start+1, length=length-1)
    return n_step

for s, l in ((9, 10), (5, 10)):
    result = count_step_numbers(s, l)
    print(f"starting digit = {s}, length={l}: {result}")

starting digit = 9, length=10: 274
starting digit = 5, length=10: 904


This only gives the number of step numbers for a specific starting digit, but by adding a loop at the top of the function we can easily get the result for all starting digits. The loop excludes 0 leading zeros are omitted.

In [2]:
def count_step_numbers(start, length=10):
    """`start` is starting digit. If None iterate over 1 to 9
    `length` is max number of digits
    """
    if start is None:
        return sum([count_step_numbers(n, length=length) for n in range(1, 10)])
    n_step = 1
    if length == 1: 
        return n_step # exactly one step number with length one
    if start > 0:
        n_step += count_step_numbers(start-1, length=length-1)
    if start < 9:
        n_step += count_step_numbers(start+1, length=length-1)
    return n_step

for l in range(1, 16):
    result = count_step_numbers(None, l)
    print(f"length={l}: {result}")

length=1: 9
length=2: 26
length=3: 58
length=4: 119
length=5: 235
length=6: 457
length=7: 881
length=8: 1694
length=9: 3250
length=10: 6236
length=11: 11957
length=12: 22939
length=13: 43992
length=14: 84408
length=15: 161913


It is easy to generalize the algorithm to only count pandigital step numbers. This could be done by passing a boolean parameter to the function for each of the ten digits. However, a key insight is that the a step number is guaranteed to be pandigital if it contains both $0$ and $9$. This is because to contain these two digits it will have had to step through all other digits.

In [3]:
def count_pandigital_step_numbers(start, length=10, seen_zero=False, seen_nine=False,):
    """`start` is starting digit. If None iterate over 1 to 9
    `length` is max number of digits
    `seen_zero` boolean to indicate that 0 is in the step number
    `seen_nine` boolean to indicate that 9 is in the step number
    """
    if start is None:
        return sum([count_pandigital_step_numbers(n,
                                                  length=length,
                                                  seen_zero=seen_zero,
                                                  seen_nine=seen_nine) 
                    for n in range(1, 10)])
    
    seen_zero = True if start == 0 else seen_zero
    seen_nine = True if start == 9 else seen_nine
    n_step = 1 if seen_zero and seen_nine else 0 # count iff step number if pandigital
    if length == 1: 
        return n_step
    if start > 0:
        n_step += count_pandigital_step_numbers(start-1, length-1, seen_zero, seen_nine)
    if start < 9:
        n_step += count_pandigital_step_numbers(start+1, length-1, seen_zero, seen_nine)
    return n_step

for l in range(1, 16):
    result = count_pandigital_step_numbers(None, l)
    print(f"length={l}: {result}")

length=1: 0
length=2: 0
length=3: 0
length=4: 0
length=5: 0
length=6: 0
length=7: 0
length=8: 0
length=9: 0
length=10: 1
length=11: 4
length=12: 18
length=13: 55
length=14: 172
length=15: 459


A weakness of the algorithm is that the execution time grows exponentially with the number of digits. However, this can be easily solved by using [memoization](https://en.wikipedia.org/wiki/Memoization) to cache function calls, which can then be utilized later

In [4]:
from functools import lru_cache

@lru_cache(maxsize=2**20)
def count_pandigital_step_numbers(start, length=10, seen_zero=False, seen_nine=False,):
    """`start` is starting digit. If None iterate over 1 to 9
    `length` is max number of digits
    `seen_zero` boolean to indicate that 0 is in the step number
    `seen_nine` boolean to indicate that 9 is in the step number
    """
    if start is None:
        return sum([count_pandigital_step_numbers(n,
                                                  length=length,
                                                  seen_zero=seen_zero,
                                                  seen_nine=seen_nine) 
                    for n in range(1, 10)])
    
    seen_zero = True if start == 0 else seen_zero
    seen_nine = True if start == 9 else seen_nine
    n_step = 1 if seen_zero and seen_nine else 0 # count iff step number if pandigital
    if length == 1: 
        return n_step
    if start > 0:
        n_step += count_pandigital_step_numbers(start-1, length-1, seen_zero, seen_nine)
    if start < 9:
        n_step += count_pandigital_step_numbers(start+1, length-1, seen_zero, seen_nine)
    return n_step

print("Result: {}".format(count_pandigital_step_numbers(None, 40)))
%timeit count_pandigital_step_numbers(None, 40)

Result: 126461847755
92.9 ns ± 2.55 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
